In [1]:
pip install IPython

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\وجدان ناصر\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [6]:
import IPython.display
IPython.display.display(IPython.display.HTML("<style>.container { width:100% !important; }</style>"))


In [7]:
#from IPython.core.display import display 

#from IPython.core.display import HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))

# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [15]:
pip install pandas  matplotlib 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\وجدان ناصر\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [17]:
pip install scikit-learn


  Using cached scikit_learn-1.6.1-cp311-cp311-win_amd64.whl.metadata (15 kB)
  Using cached scipy-1.15.2-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.6.1-cp311-cp311-win_amd64.whl (11.1 MB)
Using cached scipy-1.15.2-cp311-cp311-win_amd64.whl (41.2 MB)
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\وجدان ناصر\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [18]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [47]:
import os

# طباعة المسار الحالي
print("المسار الحالي:", os.getcwd())

# طباعة جميع الملفات في المجلد الحالي
print("الملفات في هذا المجلد:", os.listdir())
import os

# طباعة المسار الحالي
print("المسار الحالي:", os.getcwd())

# طباعة جميع الملفات في المجلد الحالي
print("الملفات في هذا المجلد:", os.listdir())


المسار الحالي: c:\Users\وجدان ناصر\OneDrive\سطح المكتب\lab\lab-natural-language-processing\your-code
الملفات في هذا المجلد: ['kg_test.csv', 'kg_train.csv', 'main.ipynb']
المسار الحالي: c:\Users\وجدان ناصر\OneDrive\سطح المكتب\lab\lab-natural-language-processing\your-code
الملفات في هذا المجلد: ['kg_test.csv', 'kg_train.csv', 'main.ipynb']


In [51]:
## Read Data for the Fraudulent Email Kaggle Challenge
#data = pd.read_csv("data/kg_train.csv", encoding='latin-1')
pd.read_csv("data/Kg_train.csv", encoding='latin-1')



# Reduce the training set to speed up development. 
# Modify for final system
data = pd.read_csv("data/Kg_train.csv", encoding='latin-1')
data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(5964, 2)


### Let's divide the training and test set into two partitions

In [52]:
# Your code
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the dataset
data = pd.read_csv("data/kg_train.csv", encoding='latin-1')

# Reduce dataset size for faster development (optional)
data = data.head(1000)

# Fill missing values with an empty string
data.fillna("", inplace=True)

# Split data into training (80%) and testing (20%)
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Print dataset shapes
print("Training set shape:", train_data.shape)
print("Testing set shape:", test_data.shape)


Training set shape: (800, 2)
Testing set shape: (200, 2)


## Data Preprocessing

In [54]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to C:\Users\وجدان
[nltk_data]     ناصر\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [55]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [63]:
pip install beautifulsoup4


   ---------------------------------------- 0.0/186.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/186.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/186.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/186.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/186.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/186.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/186.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/186.0 kB ? eta -:--:--
   -- ------------------------------------- 10.2/186.0 kB ? eta -:--:--
   -- ------------------------------------- 10.2/186.0 kB ? eta -:--:--
   -- ------------------------------------- 10.2/186.0 kB ? eta -:--:--
   ------ -------------------------------- 30.7/186.0 kB 131.3 kB/s eta 0:00:02
   ------ -------------------------------- 30.7/186.0 kB 131.3 kB/s eta 0:00:02
   ------ -------------------------------- 30.7/186.0 kB


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\وجدان ناصر\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


from bs4 import BeautifulSoup
import re

def clean_html(html):
    # Convert HTML to a BeautifulSoup object
    soup = BeautifulSoup(html, "html.parser")

    # 1️⃣ Remove JavaScript and CSS
    for script in soup(["script", "style"]):
        script.decompose()

    # 2️⃣ Remove HTML comments
    for comment in soup.findAll(text=lambda text: isinstance(text, str) and text.startswith("<!--")):
        comment.extract()

    # 3️⃣ Extract clean text
    clean_text = soup.get_text()

    # 4️⃣ Remove extra spaces and blank lines
    clean_text = re.sub(r'\s+', ' ', clean_text).strip()

    return clean_text

# 🔹 Example usage
html_code = """
<html>
<head>
    <style>body { font-size: 14px; }</style>
    <script>console.log("Hello World!");</script>
</head>
<body>
    <h1>Welcome!</h1>
    <p>This is an example of text inside HTML.</p>
    <!-- HTML Comment -->
</body>
</html>
"""

cleaned_text = clean_html(html_code)
print(cleaned_text)



In [64]:
# Your code
from bs4 import BeautifulSoup
import re

def clean_html(html):
    # Convert HTML to a BeautifulSoup object
    soup = BeautifulSoup(html, "html.parser")

    # 1️⃣ Remove JavaScript and CSS
    for script in soup(["script", "style"]):
        script.decompose()

    # 2️⃣ Remove HTML comments
    for comment in soup.findAll(text=lambda text: isinstance(text, str) and text.startswith("<!--")):
        comment.extract()

    # 3️⃣ Extract clean text
    clean_text = soup.get_text()

    # 4️⃣ Remove extra spaces and blank lines
    clean_text = re.sub(r'\s+', ' ', clean_text).strip()

    return clean_text

# 🔹 Example usage
html_code = """
<html>
<head>
    <style>body { font-size: 14px; }</style>
    <script>console.log("Hello World!");</script>
</head>
<body>
    <h1>Welcome!</h1>
    <p>This is an example of text inside HTML.</p>
    <!-- HTML Comment -->
</body>
</html>
"""

cleaned_text = clean_html(html_code)
print(cleaned_text)


Welcome! This is an example of text inside HTML.


C:\Users\وجدان ناصر\AppData\Local\Temp\ipykernel_18496\2861646948.py:14: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  for comment in soup.findAll(text=lambda text: isinstance(text, str) and text.startswith("<!--")):


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [78]:
# Your code
import re

def clean_text(text):
    # 1️⃣ Remove all special characters
    text = re.sub(r'[^A-Za-z\s]', '', text)

    # 2️⃣ Remove numbers
    text = re.sub(r'\d+', '', text)

    # 3️⃣ Remove all single characters
    text = re.sub(r'\b\w\b', '', text)

    # 4️⃣ Remove single characters from the start of words
    text = re.sub(r'\b\w\s', '', text)

    # 5️⃣ Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text).strip()

    # 6️⃣ Remove prefixed 'b' (if present at the start)
    text = re.sub(r'^b\s+', '', text)

    # 7️⃣ Convert to lowercase
    text = text.lower()

    return text

# 🔹 Example Usage
text = "Hello! This is a test b 123 with numbers, symbols @#$ and single chars a b c."
cleaned_text = clean_text(text)
print(cleaned_text)


hello this is test with numbers symbols and single chars


## Now let's work on removing stopwords
Remove the stopwords.

In [79]:
# Your code
import nltk
from nltk.corpus import stopwords
import re

# 🔹 Download stopwords (only needed once)
nltk.download('stopwords')

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))  # Load stopwords list

    # 1️⃣ Convert to lowercase
    text = text.lower()

    # 2️⃣ Remove special characters and numbers
    text = re.sub(r'[^a-z\s]', '', text)

    # 3️⃣ Tokenize (split into words)
    words = text.split()

    # 4️⃣ Remove stopwords
    filtered_words = [word for word in words if word not in stop_words]

    # 5️⃣ Rejoin words into a cleaned sentence
    cleaned_text = " ".join(filtered_words)

    return cleaned_text

# 🔹 Example Usage
text = "This is a simple example showing how to remove stopwords from a sentence!"
cleaned_text = remove_stopwords(text)
print(cleaned_text)


simple example showing remove stopwords sentence


[nltk_data] Downloading package stopwords to C:\Users\وجدان
[nltk_data]     ناصر\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [82]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [87]:
import nltk
nltk.download('punkt')


# التحقق من توفر الملفات
from nltk.tokenize import word_tokenize

try:
    word_tokenize("Test sentence")
    print("Punkt tokenizer is working correctly!")
except LookupError:
    print("Error: Punkt tokenizer is missing!")


Error: Punkt tokenizer is missing!


[nltk_data] Downloading package punkt to C:\Users\وجدان
[nltk_data]     ناصر\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [101]:
import nltk
nltk.data.path.append('C:/nltk_data')
nltk.download('punkt', download_dir='C:/nltk_data')
nltk.download('wordnet', download_dir='C:/nltk_data')
nltk.download('stopwords', download_dir='C:/nltk_data')


[nltk_data] Downloading package punkt to C:/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:/nltk_data...
[nltk_data] Downloading package stopwords to C:/nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [1]:
# Your code
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re

# 🔹 Download required resources (only needed once)
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()

    # 1️⃣ Convert text to lowercase
    text = text.lower()

    # 2️⃣ Remove special characters and numbers
    text = re.sub(r'[^a-z\s]', '', text)

    # 3️⃣ Tokenize text (split into words)
    words = word_tokenize(text)

    # 4️⃣ Lemmatize each word
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]

    # 5️⃣ Rejoin words into cleaned text
    cleaned_text = " ".join(lemmatized_words)

    return cleaned_text

# 🔹 Example Usage
text = "The cats are running faster than the mice, and the better choice is unknown."
cleaned_text = lemmatize_text(text)
print(cleaned_text)


[nltk_data] Downloading package wordnet to C:\Users\وجدان
[nltk_data]     ناصر\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\وجدان
[nltk_data]     ناصر\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\وجدان
[nltk_data]     ناصر\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\وجدان ناصر/nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\\nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\\share\\nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\\lib\\nltk_data'
    - 'C:\\Users\\وجدان ناصر\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [ ]:
import os
import nltk

# تحديد المسار يدويًا
nltk_path = os.path.expanduser('~/nltk_data')
nltk.data.path.append(nltk_path)

# تحميل البيانات
nltk.download('punkt', download_dir=nltk_path)
nltk.download('wordnet', download_dir=nltk_path)

[nltk_data] Downloading package punkt to C:\Users\وجدان
[nltk_data]     ناصر/nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to C:\Users\وجدان
[nltk_data]     ناصر/nltk_data...


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [92]:
# Your code
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

# 🔹 بيانات وهمية (تستبدل بالبيانات الفعلية الخاصة بك)
data = {
    'text': [
        "Congratulations! You won a free iPhone. Click here to claim.",
        "Hey, are we still meeting for lunch tomorrow?",
        "Reminder: Your bill is due tomorrow. Pay now to avoid penalties.",
        "Win $1000 cash now! Limited time offer, click the link below.",
        "Can you send me the report by noon?",
        "Urgent: Your account has been compromised. Reset your password now!",
        "Hey, just checking in. How are you doing?",
        "Exclusive deal: Get 50% off on your next purchase!",
        "Mom said she'll call you later today."
    ],
    'label': ['spam', 'ham', 'ham', 'spam', 'ham', 'spam', 'ham', 'spam', 'ham']
}

# ✅ تحويل البيانات إلى DataFrame
df = pd.DataFrame(data)

# ✅ تقسيم البيانات إلى رسائل "Ham" و "Spam"
ham_messages = df[df['label'] == 'ham']['text']
spam_messages = df[df['label'] == 'spam']['text']

# ✅ إنشاء نموذج Bag of Words
vectorizer = CountVectorizer(stop_words='english')  # حذف الكلمات الشائعة مثل (the, is, and)
ham_bow = vectorizer.fit_transform(ham_messages)
spam_bow = vectorizer.fit_transform(spam_messages)

# ✅ حساب تكرار الكلمات
ham_word_counts = pd.DataFrame({'word': vectorizer.get_feature_names_out(), 'count': ham_bow.toarray().sum(axis=0)})
spam_word_counts = pd.DataFrame({'word': vectorizer.get_feature_names_out(), 'count': spam_bow.toarray().sum(axis=0)})

# ✅ استخراج أكثر 10 كلمات متكررة
top_ham_words = ham_word_counts.sort_values(by='count', ascending=False).head(10)
top_spam_words = spam_word_counts.sort_values(by='count', ascending=False).head(10)

# ✅ عرض النتائج
print("\n🔹 أكثر 10 كلمات شائعة في رسائل 'Ham':")
print(top_ham_words)

print("\n🔹 أكثر 10 كلمات شائعة في رسائل 'Spam':")
print(top_spam_words)


ValueError: All arrays must be of the same length

## Extra features

In [61]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€",r"\$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

data_train.head()

NameError: name 'data_train' is not defined

## How would work the Bag of Words with Count Vectorizer concept?

In [93]:
# Your code
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

# 🔹 مجموعة نصوص (كبيانات تدريب)
documents = [
    "I love programming in Python",
    "Python is great for machine learning",
    "Machine learning is amazing",
    "I love deep learning with Python"
]

# ✅ 1️⃣ تطبيق CountVectorizer
vectorizer = CountVectorizer(stop_words="english")  # حذف الكلمات الشائعة
X = vectorizer.fit_transform(documents)

# ✅ 2️⃣ تحويل المصفوفة إلى DataFrame
df_bow = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

# ✅ 3️⃣ عرض النتيجة
print(df_bow)


   amazing  deep  great  learning  love  machine  programming  python
0        0     0      0         0     1        0            1       1
1        0     0      1         1     0        1            0       1
2        1     0      0         1     0        1            0       0
3        0     1      0         1     1        0            0       1


## TF-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [94]:
# Your code
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# 🔹 مجموعة بيانات نصية
documents = [
    "I love programming in Python",
    "Python is great for machine learning",
    "Machine learning is amazing",
    "I love deep learning with Python"
]

# ✅ 1️⃣ تحميل الـ TF-IDF Vectorizer
vectorizer = TfidfVectorizer(stop_words="english")  # إزالة الكلمات الشائعة

# ✅ 2️⃣ تحويل البيانات النصية إلى تمثيل عددي
X = vectorizer.fit_transform(documents)

# ✅ 3️⃣ طباعة شكل البيانات
print("Shape of Vectorized Dataset:", X.shape)

# ✅ 4️⃣ عرض القيم داخل جدول
df_tfidf = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
print(df_tfidf)


Shape of Vectorized Dataset: (4, 8)
    amazing      deep     great  learning      love   machine  programming  \
0  0.000000  0.000000  0.000000  0.000000  0.553492  0.000000     0.702035   
1  0.000000  0.000000  0.640655  0.408922  0.000000  0.505100     0.000000   
2  0.702035  0.000000  0.000000  0.448100  0.000000  0.553492     0.000000   
3  0.000000  0.640655  0.000000  0.408922  0.505100  0.000000     0.000000   

     python  
0  0.448100  
1  0.408922  
2  0.000000  
3  0.408922  


## And the Train a Classifier?

In [95]:
# Your code
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# 🔹 1️⃣ تحميل البيانات (مثال بسيط لرسائل Spam و Ham)
data = {
    "text": [
        "Win a free iPhone now!", "Limited offer! Claim your prize.",
        "Hello, how are you?", "Meeting at 5 PM today.",
        "Congratulations! You won $1000", "Hey, want to grab lunch?"
    ],
    "label": ["spam", "spam", "ham", "ham", "spam", "ham"]
}

df = pd.DataFrame(data)

# ✅ 2️⃣ تحويل النصوص إلى أرقام باستخدام TF-IDF
vectorizer = TfidfVectorizer(stop_words="english")
X = vectorizer.fit_transform(df["text"])
y = np.array(df["label"])

# ✅ 3️⃣ تقسيم البيانات إلى تدريب واختبار (80% تدريب، 20% اختبار)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ✅ 4️⃣ تدريب نموذج Naive Bayes
model = MultinomialNB()
model.fit(X_train, y_train)

# ✅ 5️⃣ التقييم: التنبؤ والنتائج
y_pred = model.predict(X_test)

# ✅ 6️⃣ طباعة الدقة وتقرير التصنيف
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.0
Classification Report:
               precision    recall  f1-score   support

         ham       0.00      0.00      0.00       0.0
        spam       0.00      0.00      0.00       2.0

    accuracy                           0.00       2.0
   macro avg       0.00      0.00      0.00       2.0
weighted avg       0.00      0.00      0.00       2.0



C:\Users\وجدان ناصر\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\وجدان ناصر\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\وجدان ناصر\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_class

### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to **find the most relevant features**.

For example, you can test the following options and check which of them performs better:
- Using "Bag of Words" only
- Using "TF-IDF" only
- Bag of Words + extra flags (money_mark, suspicious_words, text_len)
- TF-IDF + extra flags


You can work with teams of two persons (recommended).

In [96]:
import pandas as pd

# Load dataset from Kaggle
df = pd.read_csv("spam.csv", encoding="latin-1")

# Rename columns and keep only useful ones
df = df[['v1', 'v2']]
df.columns = ['label', 'text']

# Convert labels to numerical (spam = 1, ham = 0)
df['label'] = df['label'].map({'ham': 0, 'spam': 1})

# Display dataset info
print(df.head())
print(df['label'].value_counts())


FileNotFoundError: [Errno 2] No such file or directory: 'spam.csv'

In [97]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# Download necessary NLTK resources
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("punkt")

# Initialize stopwords and lemmatizer
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove special characters and numbers
    text = re.sub(r'[^a-z\s]', '', text)

    # Tokenize
    words = word_tokenize(text)

    # Remove stopwords and lemmatize
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]

    return " ".join(words)

# Apply preprocessing
df["clean_text"] = df["text"].apply(preprocess_text)

# Display some cleaned texts
df.head()


[nltk_data] Downloading package stopwords to C:\Users\وجدان
[nltk_data]     ناصر\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\وجدان
[nltk_data]     ناصر\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\وجدان
[nltk_data]     ناصر\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\وجدان ناصر/nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\\nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\\share\\nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\\lib\\nltk_data'
    - 'C:\\Users\\وجدان ناصر\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [98]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Split data into training & testing sets
X_train, X_test, y_train, y_test = train_test_split(df["clean_text"], df["label"], test_size=0.2, random_state=42)

# 🔹 1️⃣ Bag of Words Representation
bow_vectorizer = CountVectorizer()
X_train_bow = bow_vectorizer.fit_transform(X_train)
X_test_bow = bow_vectorizer.transform(X_test)

# 🔹 2️⃣ TF-IDF Representation
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# 🔹 3️⃣ N-grams (Bigram & Trigram)
ngram_vectorizer = CountVectorizer(ngram_range=(1, 2))  # Unigrams + Bigrams
X_train_ngram = ngram_vectorizer.fit_transform(X_train)
X_test_ngram = ngram_vectorizer.transform(X_test)


KeyError: 'clean_text'

In [99]:
# Your codedef train_and_evaluate(X_train, X_test, y_train, y_test, feature_name):
    # Train MultinomialNB 
model = MultinomialNB()
model.fit(X_train, y_train)
    
    # Predictions
y_pred = model.predict(X_test)
    
    # Print accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"🔹 Accuracy with {feature_name}: {accuracy:.4f}")
print(classification_report(y_test, y_pred))

# Evaluate each feature representation
train_and_evaluate(X_train_bow, X_test_bow, y_train, y_test, "Bag of Words")
train_and_evaluate(X_train_tfidf, X_test_tfidf, y_train, y_test, "TF-IDF")
train_and_evaluate(X_train_ngram, X_test_ngram, y_train, y_test, "N-grams")


NameError: name 'feature_name' is not defined